<a href="https://colab.research.google.com/github/adadoun/inventoryPlanningRecommendation/blob/main/PlanningRecommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview:

This notebook uses neural network to predict next week sales in order to recommender to the store the restocking strategy for each sku

## Library Import

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tqdm import tqdm
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import hashlib
import joblib

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Load the dataset
train_data = pd.read_csv('drive/MyDrive/Collab_DATA/PolarData/train_data.csv')
test_data = pd.read_csv('drive/MyDrive/Collab_DATA/PolarData/test_data.csv')

## Data Preparation for sales Prediction

In [4]:
# Prepare data for PyTorch
feature_columns = [col for col in test_data.columns if col not in ['DATE', 'QUANTITY_SOLD', 'SKU', 'CURRENT_LEVEL', 'SKU_INDEX']]


X_test = test_data[feature_columns]
y_test = test_data['QUANTITY_SOLD']
sku_test = test_data['SKU_INDEX']

# Scale features
scaler_filename = "drive/MyDrive/Collab_DATA/PolarData/scaler.save"
loaded_scaler = joblib.load(scaler_filename)
print(f"Scaler loaded from {scaler_filename}")
X_test_scaled = loaded_scaler.transform(X_test)

Scaler loaded from drive/MyDrive/Collab_DATA/PolarData/scaler.save


## Define the neural network class.

PS: In a python program, we will directly import the class written from another python module

In [5]:
class SalesNN(nn.Module):
    def __init__(self, num_features, num_skus, embedding_dim=16):
        super(SalesNN, self).__init__()
        self.sku_embedding = nn.Embedding(num_skus, embedding_dim)
        self.fc1 = nn.Linear(num_features + embedding_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 1)
        self.relu = nn.ReLU()

    def forward(self, x, sku):
        sku_emb = self.sku_embedding(sku)
        x = torch.cat((x, sku_emb), dim=1)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        return self.fc4(x).squeeze()

## Load pretrained model

In [9]:
# Load the entire model
model_load_path = 'drive/MyDrive/Collab_DATA/PolarData/NN_sales_prediction_model.pth'
loaded_model = torch.load(model_load_path)
loaded_model.eval()  # Set the model to evaluation mode

def predict(X, sku):
    with torch.no_grad():
        X = torch.FloatTensor(X)
        sku = torch.LongTensor(sku)
        predictions = loaded_model(X, sku)
    return predictions.numpy()

# Make predictions
predictions = predict(X_test_scaled, np.array(sku_test))

# Add predictions to the test dataframe
test_data['predictions'] = predictions

## Write recommendation function for stock reordering

In [11]:
def generate_weekly_recommendations(model, test_data, lead_time=2, safety_stock_factor=1.5):
    """
    Generate weekly inventory recommendations based on sales predictions.

    Args:
        model: The trained sales prediction model (not used directly in this function).
        test_data (pd.DataFrame): DataFrame containing test data with actual sales and predictions.
        lead_time (int): Number of weeks it takes for a reorder to arrive. Defaults to 2.
        safety_stock_factor (float): Factor to calculate safety stock. Defaults to 1.5.

    Returns:
        pd.DataFrame: DataFrame containing weekly recommendations for each SKU.
    """
    # Calculate prediction error
    mae = mean_absolute_error(test_data['QUANTITY_SOLD'], test_data['predictions'])

    # Create a DataFrame with actual sales, predictions, and SKUs
    results = pd.DataFrame({
        'SKU': test_data['SKU'],
        'DATE': pd.to_datetime(test_data['DATE']),
        'Actual_Sales': test_data['QUANTITY_SOLD'],
        'Predicted_Sales': test_data['predictions']
    })

    # Get the unique CURRENT_LEVEL for each SKU from the original DataFrame
    current_levels = test_data.groupby('SKU')['CURRENT_LEVEL'].first()

    # Calculate safety stock
    safety_stock = safety_stock_factor * mae * np.sqrt(lead_time)

    # Generate weekly recommendations
    recommendations = []

    # Outer loop: Iterate over each unique SKU
    for sku in results['SKU'].unique():
        sku_data = results[results['SKU'] == sku].sort_values('DATE')
        current_inventory = current_levels.get(sku, 0)

        # Initialize inventory projections for this SKU
        projected_inventory = current_inventory
        projected_inventory_without_reorder = current_inventory
        last_order_week = None

        # Inner loop: Iterate over each week for the current SKU
        for i, row in sku_data.iterrows():
            week_start = row['DATE']
            predicted_sales = row['Predicted_Sales']
            actual_sales = row['Actual_Sales']

            # Check if we need to reorder
            if projected_inventory - predicted_sales <= safety_stock:
                # Reorder is needed
                reorder_quantity = int(predicted_sales * (lead_time + 1) + safety_stock - projected_inventory)
                reorder_quantity = max(reorder_quantity, 0)  # Ensure non-negative quantity

                # Add a recommendation entry with reorder
                recommendations.append({
                    'SKU': sku,
                    'Week_Start': week_start,
                    'Reorder_Needed': 'Yes',
                    'Reorder_Quantity': reorder_quantity,
                    'Current_Inventory': projected_inventory,
                    'Predicted_Sales': predicted_sales,
                    'Actual_Sales': actual_sales,
                    'Projected_Inventory_After_Sales': projected_inventory - predicted_sales,
                    'Projected_Inventory_After_Reorder': projected_inventory - predicted_sales + reorder_quantity,
                    'Projected_Inventory_Without_Reorder': projected_inventory_without_reorder - predicted_sales
                })

                # Update inventory projections considering the reorder
                projected_inventory = projected_inventory - actual_sales + reorder_quantity
                projected_inventory_without_reorder -= actual_sales
                last_order_week = week_start
            else:
                # Reorder is not needed
                recommendations.append({
                    'SKU': sku,
                    'Week_Start': week_start,
                    'Reorder_Needed': 'No',
                    'Reorder_Quantity': 0,
                    'Current_Inventory': projected_inventory,
                    'Predicted_Sales': predicted_sales,
                    'Actual_Sales': actual_sales,
                    'Projected_Inventory_After_Sales': projected_inventory - predicted_sales,
                    'Projected_Inventory_After_Reorder': projected_inventory - predicted_sales,
                    'Projected_Inventory_Without_Reorder': projected_inventory_without_reorder - predicted_sales
                })

                # Update inventory projections without reordering
                projected_inventory -= actual_sales
                projected_inventory_without_reorder -= actual_sales

    recommendations_df = pd.DataFrame(recommendations)
    return recommendations_df

In [12]:
recommendations = generate_weekly_recommendations(loaded_model, test_data)

# Summary statistics
print("\nSummary Statistics:")
print(f"Total weeks with recommendations: {len(recommendations)}")
print(f"Weeks with reorder needed: {(recommendations['Reorder_Needed'] == 'Yes').sum()}")
print(f"Average reorder quantity when needed: {recommendations[recommendations['Reorder_Needed'] == 'Yes']['Reorder_Quantity'].mean():.2f}")


Summary Statistics:
Total weeks with recommendations: 9862
Weeks with reorder needed: 1348
Average reorder quantity when needed: 160.82


In [13]:
recommendations[recommendations.SKU == "01cb7439"]

,SKU,Week_Start,Reorder_Needed,Reorder_Quantity,Current_Inventory,Predicted_Sales,Actual_Sales,Projected_Inventory_After_Sales,Projected_Inventory_After_Reorder,Projected_Inventory_Without_Reorder
13,01cb7439,2024-03-04,No,0,381,38.330517,58,342.669483,342.669483,342.669483
14,01cb7439,2024-03-11,No,0,323,46.265121,88,276.734879,276.734879,276.734879
15,01cb7439,2024-03-18,No,0,235,58.886372,73,176.113628,176.113628,176.113628
16,01cb7439,2024-03-25,No,0,162,63.907436,98,98.092564,98.092564,98.092564
17,01cb7439,2024-04-01,Yes,235,64,76.989166,57,-12.989166,222.010834,-12.989166
18,01cb7439,2024-04-08,No,0,242,68.020126,82,173.979874,173.979874,-61.020126
19,01cb7439,2024-04-15,No,0,160,77.823914,75,82.176086,82.176086,-152.823914
20,01cb7439,2024-04-22,Yes,222,85,79.403450,86,5.596550,227.596550,-229.403450
21,01cb7439,2024-04-29,No,0,221,84.534309,92,136.465691,136.465691,-320.534309
22,01cb7439,2024-05-06,Yes,216,129,92.226845,78,36.773155,252.773155,-420.226845


## Reordering Quantity distribution by Qauntity and by SKU

In [15]:
# Distribution of reorder quantities
fig = px.histogram(recommendations[recommendations['Reorder_Needed'] == 'Yes'], x='Reorder_Quantity',
                   title='Distribution of Reorder Quantities',
                   labels={'Reorder_Quantity': 'Reorder Quantity', 'count': 'Frequency'})
fig.show()

# Reorder frequency by SKU
reorder_frequency = recommendations.groupby('SKU')['Reorder_Needed'].apply(lambda x: (x == 'Yes').mean())
fig = px.histogram(x=reorder_frequency.values,
                   title='Distribution of Reorder Frequency by SKU',
                   labels={'x': 'Proportion of Weeks Needing Reorder', 'count': 'Number of SKUs'})
fig.show()


## Example of inventory reordering over time for a given SKU

In [17]:
# Inventory levels over time for a sample SKU
sample_sku = recommendations['SKU'].iloc[0]
sku_data = recommendations[recommendations['SKU'] == "01cb7439"]

fig = go.Figure()
fig.add_trace(go.Scatter(x=sku_data['Week_Start'], y=sku_data['Current_Inventory'],
                         mode='lines', name='Current Inventory'))
fig.add_trace(go.Scatter(x=sku_data['Week_Start'], y=sku_data['Projected_Inventory_After_Sales'],
                         mode='lines', name='Projected Inventory After Sales'))
fig.add_trace(go.Scatter(x=sku_data['Week_Start'], y=sku_data['Projected_Inventory_Without_Reorder'],
                         mode='lines', name='Projected Inventory Without Reorder'))
fig.add_trace(go.Scatter(x=sku_data['Week_Start'], y=sku_data['Actual_Sales'],
                         mode='lines', name='Actual Sales'))
fig.add_trace(go.Scatter(x=sku_data['Week_Start'], y=sku_data['Predicted_Sales'],
                         mode='lines', name='Predicted Sales'))
fig.add_trace(go.Scatter(x=sku_data[sku_data['Reorder_Needed'] == 'Yes']['Week_Start'],
                         y=sku_data[sku_data['Reorder_Needed'] == 'Yes']['Current_Inventory'],
                         mode='markers', name='Reorder Points', marker=dict(color='red', size=10)))

fig.update_layout(title=f'Inventory Levels and Sales Over Time for SKU {sample_sku}',
                  xaxis_title='Week',
                  yaxis_title='Quantity',
                  legend_title='Legend')
fig.show()

## Evaluation of the recommendation program for avoiding oos

In [ ]:
# Calculate and display stockout prevention metrics
stockouts_without_reorder = (recommendations['Projected_Inventory_Without_Reorder'] <= 0).sum()
stockouts_with_reorder = (recommendations['Projected_Inventory_After_Sales'] <= 0).sum()

print(f"\nNumber of potential stockouts without reordering: {stockouts_without_reorder}")
print(f"Number of potential stockouts with reordering: {stockouts_with_reorder}")
print(f"Stockouts prevented: {stockouts_without_reorder - stockouts_with_reorder}")
print(f"Stockout prevention rate: {(stockouts_without_reorder - stockouts_with_reorder) / stockouts_without_reorder:.2%}")



Number of potential stockouts without reordering: 2325
Number of potential stockouts with reordering: 122
Stockouts prevented: 2203
Stockout prevention rate: 94.75%
